## Tranforming and gridding JULES data

Using vegetation cover from JULES: this first requires the data to be transformed from .csv to .nc files. The jules library only works on python2

In [2]:
%load_ext autoreload
%autoreload

import sys
sys.path.append('../')

import warnings
warnings.filterwarnings('ignore')

import os
from   io     import StringIO
import numpy  as np
import pandas as pd
import csv

import iris
import matplotlib.pyplot as plt
import iris.plot as iplt
import iris.quickplot as qplt
import iris.coord_categorisation
import cftime
import nc_time_axis
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs
from   libs.plot_maps    import *
from libs import jules

In [64]:
dir = '../data/jules/'

file = 'JULES-ES.1p0.vn5.4.50.GSWP3.365.S3.Monthly.'
outfile = '../data/UKESM/retrieved_codes/jul2000-jun2014/'
aList = []

for y in range(2000,2015):
    df = jules.load(dir + file + str(y) + '.nc', 'Fractional cover of each surface type')
    aList.append(df[0])

In [69]:
cube_list = iris.cube.CubeList(aList)
cubes = cube_list.concatenate_cube()

print(cubes.coord('type'))

DimCoord(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]), standard_name=None, units=Unit('1'), long_name='type')


Extracting treecover

In [75]:
tree = list(range(5))
veg = list(range(13))
print(tree)

[0, 1, 2, 3, 4]


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [76]:
index = [cubes.coord('type').points == x  for x in tree]

# This combines all the boolean arrays together. True + False = True
index = np.any(index, axis = 0)
print(index)

# Extracts just the layers we want and saves
cube = cubes[:,index]

# For just using from July 2000 to Jun 2014, but we don't have 2014...
# Just taking the off the first 6 months

cube = cube[6:-6,:,:,:].collapsed(['type'], iris.analysis.SUM)
out = outfile + 'jules_treeCover2000-2014.nc'
iris.save(cube, out)

[ True  True  True  True  True False False False False False False False
 False False False False False]


Extracting vegcover

In [ ]:
index = [cubes.coord('type').points == x  for x in veg]

# This combines all the boolean arrays together. True + False = True
index = np.any(index, axis = 0)
print(index)

# Extracts just the layers we want and saves
cube = cubes[:,index]

# For just using from July 2000 to Jun 2014, but we don't have 2014...
# Just taking the off the first 6 months

cube = cube[6:-6,:,:,:].collapsed(['type'], iris.analysis.SUM)
out = outfile + 'jules_vegcover2000-2014.nc'
iris.save(cube, out)